In [1]:
%reset
#%matplotlib qt

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import pandas as pd
import pickle
from os.path import exists
import gc

import mne
from mne.stats import spatio_temporal_cluster_test, combine_adjacency
from mne.datasets import sample
from mne.channels import find_ch_adjacency
from mne.viz import plot_compare_evokeds
from mne.time_frequency import tfr_morlet
from mne import stats
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
from mne.stats import permutation_cluster_test
from mne import io

gc.collect()

297

In [3]:
epochs_dir = 'C:\AnatArzData\YKM_data\epochs_and_evoked_allSubs'
prepro_name = "referenced"
import_type = "5Electorodes_plainEEGLAB"
fig_output_dir = "C:\\Users\Sharon\OneDrive\Desktop\AnatArzi\scripts\Python_scripts - Sharons\mycode\outputs"

## Import epochs

In [4]:
with open(f'{epochs_dir}\\{import_type}.pkl', 'rb') as file:
    [allEvents_df, allEpochs_perCond, cfg] = pickle.load(file)

cfg['sample_freq'] = 250

max_freq  =cfg['sample_freq']/2
times = cfg['times']
time0_i = np.where(times==0)[0][0]
hz_y = np.fft.rfftfreq(len(times[time0_i:]), 1.0/cfg['sample_freq'])

In [5]:
allEvents_df

,Cond_id,Subject,Vigilance,TOA,ToneHeight,TrialPosition,SamplesAmount,Epoch_eeglab_ids,TOA_cond
0,0,08,Wn,T,650,0,24,"[142, 262, 422, 682, 942, 1062, 1262, 1582, 16...",T
1,1,08,Wn,T,650,1,24,"[144, 264, 424, 684, 944, 1064, 1264, 1584, 16...",T
2,2,08,Wn,T,650,2,24,"[146, 266, 426, 686, 946, 1066, 1266, 1586, 16...",T
3,3,08,Wn,T,650,3,24,"[148, 268, 428, 688, 948, 1068, 1268, 1588, 16...",T
4,4,08,Wn,T,650,4,24,"[150, 270, 430, 690, 950, 1070, 1270, 1590, 16...",T
...,...,...,...,...,...,...,...,...,...
30235,30235,38,REM,A,5302,5,0,[],A
30236,30236,38,REM,A,5302,6,0,[],A
30237,30237,38,REM,A,5302,7,0,[],A
30238,30238,38,REM,A,5302,8,0,[],A


In [6]:
example_subject = '32'
example_epochs_filename = f"epochs_Wn_s{example_subject}_file"
data_path = 'C:\AnatArzData\Data\\reref'
subject_setfile_wake_n = data_path + '\s_32_wake_night_referenced.set'

if exists(example_epochs_filename):
    with open(example_epochs_filename, 'rb') as config_dictionary_file:
        epochs_Wn_example_sub = pickle.load(config_dictionary_file)
        #print(epochs_Wn_example_sub)
else:
    epochs_Wn_example_sub = io.read_epochs_eeglab(subject_setfile_wake_n, events=None, event_id=None,eog=(),verbose=None, uint16_codec=None)
    with open(example_epochs_filename, 'wb') as epochs_Wn_s_example_file:
        pickle.dump(epochs_Wn_example_sub, epochs_Wn_s_example_file)

montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
epochs_Wn_example_sub_piked = epochs_Wn_example_sub.pick_channels(cfg['ch_names'])
epochs_Wn_example_sub_monatged = epochs_Wn_example_sub_piked.set_montage(montage)
info = epochs_Wn_example_sub_monatged.info

In [7]:
def getEpochsPerCond(cond_df,dataset):
    df_minTrials = cond_df[(cond_df.SamplesAmount > 0)] # discard cond with 0 enough samples
    keys = (str(key) for key in df_minTrials.Cond_id)
    epochs = {str_key: dataset[str_key] for str_key in keys}
    return df_minTrials, epochs

# output: [#conds, #elect, #y_ax]
def getEvokedPerCondAndElectd(constraints,dataset,y_ax,outputType='array',tmin=-0.1):
    curr_df =allEvents_df.copy()
    # apply constraints
    for key in constraints: curr_df = curr_df[(curr_df[key] == constraints[key])]

    conds_df, epochsPerCond = getEpochsPerCond(curr_df,dataset)
    evoked_perCond_andElectd = np.zeros((len(epochsPerCond),np.size(cfg['electrodes']),len(y_ax)))

    for cond_i, cond in enumerate(epochsPerCond):
        evoked_perCond_andElectd[cond_i] = np.squeeze(np.nanmean(epochsPerCond[cond],axis=2))

    if outputType =='array':
        return conds_df, evoked_perCond_andElectd
    elif outputType =='mne':
        # Epochs array shapr should be: (len(epochs_samps_per_cond),len(electrodes),len(y_ax))
        mne_epochs = mne.EpochsArray(evoked_perCond_andElectd, info,tmin=tmin,baseline=baseline)
        return conds_df, mne_epochs ## TODO: this is epochs. Maybe we need evoked.

In [8]:
def applyDesign(ax,title=''):
    ax.set_facecolor('silver')
    ax.set_title(title)
    ax.legend(loc = 'upper right',prop={'size': 10})
    ax.axvline(x=0,color='gray', linestyle='--',label ="_nolegend_")
    ax.axhline(y=0, color='gray', linestyle='-',label ="_nolegend_")
    ax.set_ylabel('magnitude')
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_xlabel('Time (ms)')

## Time-freq functions

In [9]:
cond_1 = {'TOA_cond':'T','Vigilance':'Wn'}
cond_2 = {'TOA_cond':'T','Vigilance':'N2'}
contrast_name = "allSubs_T_WnvsN3"

#### Parameters:
# ==================
# zscore baseline
## Tail is 0, so the statistic is thresholded on both sides of the distribution.
baseline = (None, 0) ## For timefreq analysis
p_value = 0.05 # default 0.05 # for clusters
decim = 1# default 2 for testing.. For reals - 1
n_permutations=1000 # default 1k for testing. For reals - 10K
min_freq = 4
freqs = np.arange(min_freq,80, 1)  # define frequencies of interest
n_cycles = freqs / min_freq # different number of cycle per frequency

tail = 0 # 0 = two-tailed test (for clusters (as we want positive and negative clusters))
p_value_pixels = 0.05

In [20]:
adjacency, ch_names = find_ch_adjacency(epochs_Wn_example_sub.info, ch_type='eeg')
print(type(adjacency))  # it's a sparse matrix!

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 5
<class 'scipy.sparse.csr.csr_matrix'>
